In [67]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.shuffle.useOldFetchProtocol','true'). \
config("spark.sql.warehouse.dir", f"/user/itv021558/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [68]:
raw_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/public/trendytech/datasets/accepted_2007_to_2018Q4.csv")

In [69]:
raw_df.createOrReplaceTempView("lending_club_data")

In [5]:
spark.sql("select * from lending_club_data")

+--------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+------------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------------+-----------+------------------+--------------------+---------------------+--------------+--------+--------------------+------------------------+---------------------+------------------------------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+----------------+------------+------------------+------------------+--------------+----------------+--------------------+---------+---------------+-----------------+--------------+--------------------------+---------------+----------------------+-----------------------+------------------------+----------------------+----------------+----------------+------------------+-------------------+---------------------+--------------------------------+----------------------------------+-----------------------------------+-------------+-------------+---------------+---------------+-------------+---------------+-------------------+-----------------+-----------------------+---------------+------------+--------------------+------------------------------------------+------------------------------+----------------------------+-------------------+--------------------+-------------------------+-----------------+---------------+-----------------+---------------------+---------------+
|      id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|           emp_title|emp_length|home_ownership|annual_inc|verification_status| issue_d|loan_status|pymnt_plan|                 url|desc|           purpose|               title|zip_code|addr_state|  dti|delinq_2yrs|earliest_cr_line|fico_range_low|fico_range_high|inq_last_6mths|mths_since_last_delinq|mths_since_last_record|open_acc|pub_rec|revol_bal|revol_util|total_acc|initial_list_status|out_prncp|out_prncp_inv|       total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|last_pymnt_d|last_pymnt_amnt|next_pymnt_d|last_credit_pull_d|last_fico_range_high|last_fico_range_low|collections_12_mths_ex_med|mths_since_last_major_derog|policy_code|application_type|annual_inc_joint|dti_joint|verification_status_joint|acc_now_delinq|tot_coll_amt|tot_cur_bal|open_acc_6m|open_act_il|open_il_12m|open_il_24m|mths_since_rcnt_il|total_bal_il|il_util|open_rv_12m|open_rv_24m|max_bal_bc|all_util|total_rev_hi_lim|inq_fi|total_cu_tl|inq_last_12m|acc_open_past_24mths|avg_cur_bal|bc_open_to_buy|bc_util|chargeoff_within_12_mths|delinq_amnt|mo_sin_old_il_acct|mo_sin_old_rev_tl_op|mo_sin_rcnt_rev_tl_op|mo_sin_rcnt_tl|mort_acc|mths_since_recent_bc|mths_since_recent_bc_dlq|mths_since_recent_inq|mths_since_recent_revol_delinq|num_accts_ever_120_pd|num_actv_bc_tl|num_actv_rev_tl|num_bc_sats|num_bc_tl|num_il_tl|num_op_rev_tl|num_rev_accts|num_rev_t

In [13]:
spark.sql("select loan_amnt, total_pymnt from lending_club_data ")

+---------+------------------+
|loan_amnt|       total_pymnt|
+---------+------------------+
|   3600.0| 4421.723916800001|
|  24700.0|          25679.66|
|  20000.0|22705.924293878397|
|  35000.0|          31464.01|
|  10400.0|           11740.5|
|  11950.0|  13708.9485297572|
|  20000.0|   21393.800000011|
|  20000.0|   21538.508976797|
|  10000.0|  10998.9715749644|
|   8000.0|   8939.5805031401|
|  22400.0|          19275.33|
|  16000.0|          13768.04|
|   1400.0|    1575.160697674|
|  18000.0|           9452.74|
|  28000.0|29939.017729337003|
|   9600.0|10636.098427692801|
|  25000.0|          26224.23|
|  18000.0|          18387.22|
|  23000.0|          17900.14|
|   8650.0|           9190.49|
+---------+------------------+
only showing top 20 rows

In [70]:
from pyspark.sql.functions import sha2, concat_ws

In [71]:
new_df = raw_df.withColumn("name_sha2",sha2(concat_ws("||",*["emp_title","emp_length","home_ownership","annual_inc","zip_code","addr_state","grade","sub_grade","verification_status"]),256))

In [15]:
new_df

+--------+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------------+----------+--------------+----------+-------------------+--------+-----------+----------+--------------------+----+------------------+--------------------+--------+----------+-----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+------------------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------------+-----------+------------------+--------------------+---------------------+--------------+--------+--------------------+------------------------+---------------------+------------------------------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+----------------+------------+------------------+------------------+--------------+----------------+--------------------+---------+---------------+-----------------+--------------+--------------------------+---------------+----------------------+-----------------------+------------------------+----------------------+----------------+----------------+------------------+-------------------+---------------------+--------------------------------+----------------------------------+-----------------------------------+-------------+-------------+---------------+---------------+-------------+---------------+-------------------+-----------------+-----------------------+---------------+------------+--------------------+------------------------------------------+------------------------------+----------------------------+-------------------+--------------------+-------------------------+-----------------+---------------+-----------------+---------------------+---------------+--------------------+
|      id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|      term|int_rate|installment|grade|sub_grade|           emp_title|emp_length|home_ownership|annual_inc|verification_status| issue_d|loan_status|pymnt_plan|                 url|desc|           purpose|               title|zip_code|addr_state|  dti|delinq_2yrs|earliest_cr_line|fico_range_low|fico_range_high|inq_last_6mths|mths_since_last_delinq|mths_since_last_record|open_acc|pub_rec|revol_bal|revol_util|total_acc|initial_list_status|out_prncp|out_prncp_inv|       total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|last_pymnt_d|last_pymnt_amnt|next_pymnt_d|last_credit_pull_d|last_fico_range_high|last_fico_range_low|collections_12_mths_ex_med|mths_since_last_major_derog|policy_code|application_type|annual_inc_joint|dti_joint|verification_status_joint|acc_now_delinq|tot_coll_amt|tot_cur_bal|open_acc_6m|open_act_il|open_il_12m|open_il_24m|mths_since_rcnt_il|total_bal_il|il_util|open_rv_12m|open_rv_24m|max_bal_bc|all_util|total_rev_hi_lim|inq_fi|total_cu_tl|inq_last_12m|acc_open_past_24mths|avg_cur_bal|bc_open_to_buy|bc_util|chargeoff_within_12_mths|delinq_amnt|mo_sin_old_il_acct|mo_sin_old_rev_tl_op|mo_sin_rcnt_rev_tl_op|mo_sin_rcnt_tl|mort_acc|mths_since_recent_bc|mths_since_recent_bc_dlq|mths_since_recent_inq|mths_since_recent_revol_delinq|num_accts_ever_120_pd|num_actv_bc_tl|num_actv_rev_tl|num_bc_sats|num_bc_tl|num_il_tl|num_op_rev_tl|nu

In [72]:
new_df.createOrReplaceTempView("newtable")

In [17]:
spark.sql("select count(*)from newtable")

+--------+
|count(1)|
+--------+
| 2260701|
+--------+

In [18]:
spark.sql("select count(distinct(name_sha2))from newtable")

+-------------------------+
|count(DISTINCT name_sha2)|
+-------------------------+
|                  2257384|
+-------------------------+

In [19]:
spark.sql("select name_sha2, count(*) as total_cnt from newtable group by name_sha2 having total_cnt > 1 order by total_cnt desc")

+--------------------+---------+
|           name_sha2|total_cnt|
+--------------------+---------+
|e3b0c44298fc1c149...|       33|
|e4c167053d5418230...|        5|
|3f87585a20f702838...|        4|
|ad8e5d384dae17e06...|        4|
|76b577467eda5bdbc...|        4|
|c92062bb371842b3d...|        3|
|059d401bb603d9a80...|        3|
|d9ce4046daf599732...|        3|
|3ae415acd6bbfaac1...|        3|
|291ca1b09ef11ca3e...|        3|
|5d52e7773cb0efff3...|        3|
|d3731bad3e6a82515...|        3|
|e7d8d16928817ec8f...|        3|
|22593a1870543b2db...|        3|
|53789bea7edc660ed...|        3|
|498bb6b1f0099cb47...|        3|
|a2356725aa9da92c6...|        3|
|4ab6205de571ccb7b...|        3|
|c563428cb58da48ff...|        3|
|f54295a60946dedad...|        3|
+--------------------+---------+
only showing top 20 rows

In [22]:
spark.sql("select * from newtable where name_sha2 like'e3b0c44298fc1c149%'")

+--------------------+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+-------+-----------+----------+----+----+-------+-----+--------+----------+----+-----------+----------------+--------------+---------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------+-------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------------------+-----------+--------------+-------+------------------------+-----------+------------------+--------------------+---------------------+--------------+--------+--------------------+------------------------+---------------------+------------------------------+---------------------+--------------+---------------+-----------+---------+---------+-------------+-------------+-------------------+--------+----------------+------------+------------------+------------------+--------------+----------------+--------------------+---------+---------------+-----------------+--------------+--------------------------+---------------+----------------------+-----------------------+------------------------+----------------------+----------------+----------------+------------------+-------------------+---------------------+--------------------------------+----------------------------------+-----------------------------------+-------------+-------------+---------------+---------------+-------------+---------------+-------------------+-----------------+-----------------------+---------------+------------+--------------------+------------------------------------------+------------------------------+----------------------------+-------------------+--------------------+-------------------------+-----------------+---------------+-----------------+---------------------+---------------+--------------------+
|                  id|member_id|loan_amnt|funded_amnt|funded_amnt_inv|term|int_rate|installment|grade|sub_grade|emp_title|emp_length|home_ownership|annual_inc|verification_status|issue_d|loan_status|pymnt_plan| url|desc|purpose|title|zip_code|addr_state| dti|delinq_2yrs|earliest_cr_line|fico_range_low|fico_range_high|inq_last_6mths|mths_since_last_delinq|mths_since_last_record|open_acc|pub_rec|revol_bal|revol_util|total_acc|initial_list_status|out_prncp|out_prncp_inv|total_pymnt|total_pymnt_inv|total_rec_prncp|total_rec_int|total_rec_late_fee|recoveries|collection_recovery_fee|last_pymnt_d|last_pymnt_amnt|next_pymnt_d|last_credit_pull_d|last_fico_range_high|last_fico_range_low|collections_12_mths_ex_med|mths_since_last_major_derog|policy_code|application_type|annual_inc_joint|dti_joint|verification_status_joint|acc_now_delinq|tot_coll_amt|tot_cur_bal|open_acc_6m|open_act_il|open_il_12m|open_il_24m|mths_since_rcnt_il|total_bal_il|il_util|open_rv_12m|open_rv_24m|max_bal_bc|all_util|total_rev_hi_lim|inq_fi|total_cu_tl|inq_last_12m|acc_open_past_24mths|avg_cur_bal|bc_open_to_buy|bc_util|chargeoff_within_12_mths|delinq_amnt|mo_sin_old_il_acct|mo_sin_old_rev_tl_op|mo_sin_rcnt_rev_tl_op|mo_sin_rcnt_tl|mort_acc|mths_since_recent_bc|mths_since_recent_bc_dlq|mths_since_recent_inq|mths_since_recent_revol_delinq|num_accts_ever_120_pd|num_actv_bc_tl|num_actv_rev_tl|num_bc_sats|num_bc_tl|num_il_tl|num_op_rev_tl|num_rev_accts|num_rev_tl_bal_gt_0|num_sats|num_tl_120dpd_2m|num_tl_30dpd|num_tl_90g_dpd_24m|num_tl_op_past_12m|pct

In [39]:
spark.sql("""select name_sha2 as member_id, emp_title, emp_length, home_ownership, annual_inc, 
addr_state, zip_code, 'USA' as country, grade, sub_grade, verification_status, tot_hi_cred_lim, 
application_type,annual_inc_joint, verification_status_joint from newtable
""").repartition(1) \
.write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.save("/user/itv021558/lendingclubproject/raw/customers_data_csv")

In [40]:
customers_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/user/itv021558/lendingclubproject/raw/customers_data_csv")

In [41]:
customers_df

+--------------------+--------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+
|           member_id|           emp_title|emp_length|home_ownership|annual_inc|addr_state|zip_code|country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|annual_inc_joint|verification_status_joint|
+--------------------+--------------------+----------+--------------+----------+----------+--------+-------+-----+---------+-------------------+---------------+----------------+----------------+-------------------------+
|a4ec00ba67fadf2fe...|  Executive Director|  < 1 year|      MORTGAGE|   81000.0|        PA|   152xx|    USA|    C|       C5|           Verified|       229529.0|      Individual|            null|                     null|
|4f7a9e6ffaacd5da2...|                null|      null|          RENT|   46000.0|        CA|   941xx|    USA|    B|       B5|    Source Verified|        24462.0|      Individual|            null|                     null|
|e935a4c27fc78ae61...|            Swingman|   6 years|      MORTGAGE|   80000.0|        IN|   467xx|    USA|    A|       A4|       Not Verified|       237978.0|      Individual|            null|                     null|
|2d32004bd5e1dc3c3...|      Bindery worker| 10+ years|          RENT|   60000.0|        CA|   917xx|    USA|    D|       D5|    Source Verified|        62348.0|      Individual|            null|                     null|
|f7116b7f7546a7952...|                null|      null|           OWN|   40000.0|        OH|   440xx|    USA|    D|       D2|           Verified|        91325.0|      Individual|            null|                     null|
|d3aa3a3c95eca5631...|              banker|    1 year|           OWN|   35000.0|        AZ|   857xx|    USA|    E|       E5|           Verified|        35101.0|      Individual|            null|                     null|
|fc8a2e046eaaba02d...|    Registered Nurse| 10+ years|      MORTGAGE|   85000.0|        NC|   284xx|    USA|    C|       C2|    Source Verified|       237478.0|      Individual|            null|                     null|
|577ae670ac2ec7ed3...|             Chemist|    1 year|           OWN|   43908.0|        MO|   631xx|    USA|    B|       B2|    Source Verified|       100326.0|      Individual|            null|                     null|
|d3792868819649ba9...|Supervisory Instr...|  < 1 year|      MORTGAGE|  107000.0|        WV|   254xx|    USA|    D|       D1|    Source Verified|       146840.0|      Individual|            null|                     null|
|6d3a5a422261348b3...|Customer Service ...|   7 years|           OWN|   45000.0|        MI|   490xx|    USA|    E|       E1|    Source Verified|       176007.0|      Individual|            null|                     null|
|7435a5a4b2b825a20...|Chief Financial F...|   2 years|          RENT|  195000.0|        NE|   688xx|    USA|    D|       D5|    Source Verified|       218103.0|      Individual|            null|                     null|
|2a1dd1ae5b20c4dee...|      branch manager|   5 years|      MORTGAGE|  190000.0|        CA|   919xx|    USA|    C|       C1|    Source Verified|       478116.0|      Individual|            null|                     null|
|8d203efd7c1af010c...|    Security Officer|    1 year|          RENT|   33000.0|        MN|   553xx|    USA|    B|       B2|    Source Verified|        58043.0|      Individual|            null|                     null|
|c266c3c131861bb8c...|                null|      null|          RENT|   35000.0|        VA|   223xx|    USA|    B|       B3|           Verified|        17100.0|      Individual|            null|                     null|
|c7b46f35c566b6d56...|  Station Technician|   3 years|      MORTGAGE|   67000.0|        HI|   967xx|    USA|    D|       D5|           Verified|        52615.0|      Individual|            null|                     null|
|9284503d1d6dcd23b...|

In [52]:
spark.sql("""select id as loan_id, name_sha2 as member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,
 loan_status,purpose,title from newtable""").repartition(1) \
.write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.save("/user/itv021558/lendingclubproject/raw/loans_data_csv")

In [53]:
loans_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/user/itv021558/lendingclubproject/raw/loans_data_csv")

In [54]:
loans_df

+--------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+------------------+--------------------+
| loan_id|           member_id|loan_amnt|funded_amnt|     term|int_rate|installment| issue_d|loan_status|           purpose|               title|
+--------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+------------------+--------------------+
|68407277|6d5091b3fcaaeb4ea...|   3600.0|     3600.0|36 months|   13.99|     123.03|Dec-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|68355089|b5e7938b0a2da4cea...|  24700.0|    24700.0|36 months|   11.99|     820.28|Dec-2015| Fully Paid|    small_business|            Business|
|68341763|91060b858433e8a61...|  20000.0|    20000.0|60 months|   10.78|     432.66|Dec-2015| Fully Paid|  home_improvement|                null|
|66310712|cab1fa9f533688b0a...|  35000.0|    35000.0|60 months|   14.85|      829.9|Dec-2015|    Current|debt_consolidation|  Debt consolidation|
|68476807|f74e401c1ab0adf78...|  10400.0|    10400.0|60 months|   22.45|     289.91|Dec-2015| Fully Paid|    major_purchase|      Major purchase|
|68426831|8aef4bb29d609d8d6...|  11950.0|    11950.0|36 months|   13.44|     405.18|Dec-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|68476668|538b4653da3b1e814...|  20000.0|    20000.0|36 months|    9.17|     637.58|Dec-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|67275481|b24d55f21390533c5...|  20000.0|    20000.0|36 months|    8.49|     631.26|Dec-2015| Fully Paid|    major_purchase|      Major purchase|
|68466926|1035c5401b0ca76d0...|  10000.0|    10000.0|36 months|    6.49|     306.45|Dec-2015| Fully Paid|       credit_card|Credit card refin...|
|68616873|cb0f1777593e77909...|   8000.0|     8000.0|36 months|   11.48|     263.74|Dec-2015| Fully Paid|       credit_card|Credit card refin...|
|68356421|a962f4d59caec5fa1...|  22400.0|    22400.0|60 months|   12.88|      508.3|Dec-2015|    Current|debt_consolidation|  Debt consolidation|
|68426545|e7592ab57b3afd9f1...|  16000.0|    16000.0|60 months|   12.88|     363.07|Dec-2015|    Current|debt_consolidation|  Debt consolidation|
|68338832|603afa9d1be879b7b...|   1400.0|     1400.0|36 months|   12.88|       47.1|Dec-2015| Fully Paid|             other|                null|
|66624733|9fe2d59ddf2a4f37e...|  18000.0|    18000.0|60 months|   19.48|      471.7|Dec-2015|Charged Off|debt_consolidation|  Debt consolidation|
|68466961|23857480ccf555ce4...|  28000.0|    28000.0|36 months|    6.49|     858.05|Dec-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|68354783|7c69d5f36fdabf6e1...|   9600.0|     9600.0|36 months|    7.49|     298.58|Dec-2015| Fully Paid|       credit_card|Credit card refin...|
|68466916|08bf9e080503b0113...|  25000.0|    25000.0|36 months|    7.49|     777.55|Dec-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|68577849|1cafb05aa6c894c30...|  18000.0|    18000.0|60 months|   11.99|     400.31|Dec-2015| Fully Paid|debt_consolidation|  Debt consolidation|
|68506798|f2c4010f700d8c9c4...|  23000.0|    23000.0|60 months|    8.49|     471.77|Dec-2015|    Current|       credit_card|Credit card refin...|
|68495092|05ad4aed7c393035e...|   8650.0|     8650.0|36 months|   19.89|     320.99|Dec-2015| Fully Paid|debt_consolidation|  Debt consolidation|
+--------+--------------------+---------+-----------+---------+--------+-----------+--------+-----------+------------------+--------------------+
only showing top 20 rows

In [74]:
spark.sql("""select id as loan_id,total_rec_prncp,total_rec_int,total_rec_late_fee,total_pymnt,last_pymnt_amnt,
last_pymnt_d,next_pymnt_d from newtable""").repartition(1) \
.write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.save("/user/itv021558/lendingclubproject/raw/loans_repay_data_csv")

In [75]:
loans_repay_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/user/itv021558/lendingclubproject/raw/loans_repay_data_csv")

In [76]:
loans_repay_df

+--------+---------------+-------------+------------------+------------------+---------------+------------+------------+
| loan_id|total_rec_prncp|total_rec_int|total_rec_late_fee|       total_pymnt|last_pymnt_amnt|last_pymnt_d|next_pymnt_d|
+--------+---------------+-------------+------------------+------------------+---------------+------------+------------+
|69642551|        3246.03|       990.53|               0.0|           4965.02|         326.35|    Feb-2017|        null|
|69336966|        12000.0|       1080.5|               0.0|           13080.5|        9501.81|    Nov-2016|        null|
|69226186|        28000.0|      3049.36|               0.0|31049.361806989902|        9306.81|    Mar-2018|        null|
|69084024|        16500.0|       900.32|               0.0|          17400.32|       14085.82|    Aug-2016|        null|
|69422635|        21000.0|      4452.42|               0.0|  25452.4171310098|       12705.78|    Jun-2018|        null|
|69542436|        1220.15|      1252.23|               0.0|           4533.19|         497.94|    Jul-2016|        null|
|69492501|        29000.0|      5405.32|              47.8|          34453.12|         771.08|    Mar-2019|    Apr-2019|
|69522527|        33000.0|      2092.69|               0.0|          35092.69|          65.34|    Jul-2016|        null|
|68781653|        18000.0|      4167.34|               0.0|  22167.3434977632|       13021.32|    Jan-2018|        null|
|69371334|        16000.0|       1140.7|               0.0|  17140.7005770789|       13910.33|    Dec-2016|        null|
|68714098|        15000.0|      1665.34|               0.0|16665.335109603002|         463.01|    Jan-2019|        null|
|69622505|       14993.49|     15587.45|               0.0|          30580.94|         925.15|    Nov-2018|    Apr-2019|
|69084020|        8191.71|      2319.93|               0.0|          11211.64|         403.64|    Mar-2018|        null|
|69552651|       13221.14|      7957.71|               0.0|          21178.85|         558.32|    Mar-2019|    Apr-2019|
|69084037|         7200.0|        983.7|               0.0| 8183.703417215899|        4153.17|    Jul-2017|        null|
|69612464|        2189.97|       342.22|               0.0|           2532.19|          78.91|    Oct-2018|        null|
|69562380|       11481.73|     10842.62|               0.0|          22324.35|         588.13|    Mar-2019|    Apr-2019|
|69422542|         7200.0|       539.02|               0.0|   7739.0247995761|         194.79|    Jan-2017|        null|
|69652408|         1500.0|       317.81|               0.0|1817.8054757467999|         246.22|    Apr-2018|        null|
|68756432|       17271.58|      6819.29|               0.0|          24090.87|         634.47|    Mar-2019|    Apr-2019|
+--------+---------------+-------------+------------------+------------------+---------------+------------+------------+
only showing top 20 rows

In [78]:
spark.sql("""select name_sha2 as member_id,delinq_2yrs,delinq_amnt,pub_rec,pub_rec_bankruptcies,inq_last_6mths,
total_rec_late_fee, mths_since_last_delinq, mths_since_last_record from newtable""").repartition(1) \
.write \
.option("header",True) \
.format("csv") \
.mode("overwrite") \
.save("/user/itv021558/lendingclubproject/raw/loans_defaulters_data_csv")

In [79]:
loans_repay_df = spark.read \
.format("csv") \
.option("inferSchema","true") \
.option("header","true") \
.load("/user/itv021558/lendingclubproject/raw/loans_defaulters_data_csv")

In [82]:
loans_repay_df

+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|           member_id|delinq_2yrs|delinq_amnt|pub_rec|pub_rec_bankruptcies|inq_last_6mths|total_rec_late_fee|mths_since_last_delinq|mths_since_last_record|
+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
|5d7676571dee53d06...|        1.0|        0.0|    1.0|                 1.0|           1.0|               0.0|                   4.0|                 107.0|
|31fd1ef036c5caf26...|        0.0|        0.0|    0.0|                 0.0|           3.0|               0.0|                  null|                  null|
|9b48253ca5848fa78...|        1.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  20.0|                  null|
|d6b8f2e32be148721...|        0.0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  28.0|                  null|
|0a81ad556e20bcdb9...|        0.0|        0.0|    0.0|                 0.0|           2.0|               0.0|                  null|                  null|
|843b995d39160ddab...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  26.0|                  null|
|04a8bd7c4e1407ba8...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  null|                  null|
|3e9a89fa96988af68...|        2.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                   6.0|                  null|
|394a8db0bd3bcf7de...|        1.0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  10.0|                  null|
|09cea8d2b6f1272f7...|        0.0|        0.0|    0.0|                 0.0|           5.0|               0.0|                  null|                  null|
|bc13b4bf12b2fb93f...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  null|                  null|
|dcc3cce84164a1a4f...|        0.0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  null|                  null|
|a8c2898592c8c34ea...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  27.0|                  null|
|2ab161847c8ce1f3e...|        1.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                   3.0|                  null|
|9886b3db401c1d460...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  null|                  null|
|7b62e3c918a7f9eb8...|        1.0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  20.0|                  null|
|698b533486c4900d4...|        1.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  16.0|                  null|
|0b07196bd0485e4c2...|        0.0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  33.0|                  null|
|92e61f312d73c874d...|        0.0|        0.0|    0.0|                 0.0|           0.0|               0.0|                  52.0|                  null|
|a2eb7934b6490e443...|        0.0|        0.0|    0.0|                 0.0|           1.0|               0.0|                  null|                  null|
+--------------------+-----------+-----------+-------+--------------------+--------------+------------------+----------------------+----------------------+
only showing top 20 rows

In [83]:
spark.stop()